# Планирование эксперимента. Лабораторная работа №1

Стандартная преамбула для улучшения визуализации.

In [408]:
# The usual preamble
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Make the graphs a bit prettier, and bigger
plt.show(block=True)
plt.style.use('bmh')

plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['font.family'] = 'sans-serif'

Загружаем датасет.

In [409]:
dataset = pd.read_csv("athlete_events.csv")
dataset[:5]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


Для лабораторной работы будем использовать ряд возрастов атлетов.

In [410]:
ages = dataset['Age']
ages = ages[ages.notnull()]
ages.unique()

array([24., 23., 34., 21., 25., 27., 31., 33., 18., 26., 22., 30., 32.,
       28., 54., 20., 17., 43., 47., 29., 41., 45., 49., 53., 57., 19.,
       38., 35., 16., 37., 15., 42., 46., 40., 36., 14., 39., 48., 52.,
       44., 55., 50., 71., 63., 51., 58., 13., 60., 75., 65., 56., 64.,
       68., 84., 12., 72., 59., 61., 70., 74., 62., 67., 69., 73., 66.,
       11., 76., 88., 96., 80., 10., 81., 77., 97.])

Разбиваем значения на 6 равных интервалов и вычисляем эмпирическую частоту попадания в каждый интервал.

In [411]:
df = pd.DataFrame({'Эмп. частота': ages})
ranges = [0,10,20,30,40,50,60,70,80,90,100]
table = df.groupby(pd.cut(df['Эмп. частота'], 6)).count()
table.index.name = 'Интервалы'
table

,Эмп. частота
Интервалы,
"(9.913, 24.5]",131134
"(24.5, 39.0]",121597
"(39.0, 53.5]",7616
"(53.5, 68.0]",1106
"(68.0, 82.5]",183
"(82.5, 97.0]",6


Вычислим среднее значение на каждом интервале.

In [412]:
midpoints = pd.DataFrame({'Ср. на интевале': [val.mid for val in table.index]}, index=table.index)
table = table.add(midpoints, fill_value=0)
table

,Ср. на интевале,Эмп. частота
Интервалы,,
"(9.913, 24.5]",17.2065,131134.0
"(24.5, 39.0]",31.7500,121597.0
"(39.0, 53.5]",46.2500,7616.0
"(53.5, 68.0]",60.7500,1106.0
"(68.0, 82.5]",75.2500,183.0
"(82.5, 97.0]",89.7500,6.0


Вычисляем эмпирическую вероятность попадания в интервал.

In [413]:
probabs = pd.DataFrame({'Эмп. вероятность': [val / table['Эмп. частота'].sum() 
                                             for val in table['Эмп. частота']]}, index=table.index)
table = table.add(probabs, fill_value=0)
table

,Ср. на интевале,Эмп. вероятность,Эмп. частота
Интервалы,,,
"(9.913, 24.5]",17.2065,0.501196,131134.0
"(24.5, 39.0]",31.7500,0.464746,121597.0
"(39.0, 53.5]",46.2500,0.029108,7616.0
"(53.5, 68.0]",60.7500,0.004227,1106.0
"(68.0, 82.5]",75.2500,0.000699,183.0
"(82.5, 97.0]",89.7500,0.000023,6.0


Рассчитываем значения функции распределения вероятности.

In [414]:
fx_df = pd.DataFrame({'F(x)': [table['Эмп. вероятность'][:i].sum()
                               for i, val in enumerate(table['Эмп. вероятность'])]}, index=table.index)
table = table.add(fx_df, fill_value=0)
table

,F(x),Ср. на интевале,Эмп. вероятность,Эмп. частота
Интервалы,,,,
"(9.913, 24.5]",0.000000,17.2065,0.501196,131134.0
"(24.5, 39.0]",0.501196,31.7500,0.464746,121597.0
"(39.0, 53.5]",0.965942,46.2500,0.029108,7616.0
"(53.5, 68.0]",0.995050,60.7500,0.004227,1106.0
"(68.0, 82.5]",0.999278,75.2500,0.000699,183.0
"(82.5, 97.0]",0.999977,89.7500,0.000023,6.0


In [419]:
xipi = pd.DataFrame({'Xi*Pi': [table['Ср. на интевале'][i] * val
                               for i, val in enumerate(table['Эмп. вероятность'])]}, index=table.index)
table = table.add(xipi, fill_value=0)
table

,F(x),xi*pi,Ср. на интевале,Эмп. вероятность,Эмп. частота
Интервалы,,,,,
"(9.913, 24.5]",0.000000,8.623834,17.2065,0.501196,131134.0
"(24.5, 39.0]",0.501196,14.755677,31.7500,0.464746,121597.0
"(39.0, 53.5]",0.965942,1.346267,46.2500,0.029108,7616.0
"(53.5, 68.0]",0.995050,0.256799,60.7500,0.004227,1106.0
"(68.0, 82.5]",0.999278,0.052632,75.2500,0.000699,183.0
"(82.5, 97.0]",0.999977,0.002058,89.7500,0.000023,6.0


In [421]:
xi2pi = pd.DataFrame({'(Xi^2)*Pi': [(table['Ср. на интевале'][i] ** 2) * val
                               for i, val in enumerate(table['Эмп. вероятность'])]}, index=table.index)
table = table.add(xi2pi, fill_value=0)
table

,(Xi^2)*Pi,F(x),xi*pi,Ср. на интевале,Эмп. вероятность,Эмп. частота
Интервалы,,,,,,
"(9.913, 24.5]",148.385999,0.000000,8.623834,17.2065,0.501196,131134.0
"(24.5, 39.0]",468.492734,0.501196,14.755677,31.7500,0.464746,121597.0
"(39.0, 53.5]",62.264850,0.965942,1.346267,46.2500,0.029108,7616.0
"(53.5, 68.0]",15.600562,0.995050,0.256799,60.7500,0.004227,1106.0
"(68.0, 82.5]",3.960560,0.999278,0.052632,75.2500,0.000699,183.0
"(82.5, 97.0]",0.184719,0.999977,0.002058,89.7500,0.000023,6.0


In [ ]:
var = pd.DataFrame({'D': [(table['Ср. на интевале'][i] ** 2) * val
                               for i, val}, index=table.index)
table.add(var, fill_value=0)
# table